In [127]:
import pandas as pd
import numpy as np
import os

# uncomment this next section to create your own data.csv file for ETFs, as Github does not allow pushing files this large
                    # https://stackoverflow.com/questions/44946763/pandas-dataframe-correlation-on-part-date
df = pd.DataFrame()
dirs = os.listdir(".\ETFs")
print(dirs)
i = 0
for d in dirs:
    if d != "Other":
        files = os.listdir(f".\ETFs\{d}")
        for f in files:
            a = pd.read_csv(f".\ETFs\{d}\{f}")
            a = a[a.columns[:-1]]
            a["Sector"]=d
            a["ETF"]=f.split(".")[0].upper()
            a = a[["Date","ETF","Sector"]+list(a.columns[1:-2])]
            #print(a)
            df = pd.concat([df,a],ignore_index=True)

df.to_csv("data.csv") # so that the computation wouldn't take this long next time

ETFdf = pd.read_csv("data.csv")
ETFdf = ETFdf[ETFdf.columns[1:]]
ETFdf = ETFdf[(ETFdf["Date"]>="2008-01-25") & (ETFdf["Date"]<="2016-11-22")] # the data for the other dataset is between these dates
ETFdf
#ETFdf[ETFdf["Date"]=="YYYY-MM-DD"]# return stock data on a specific date

Rdtdf = pd.read_csv("reddit_data.csv")
Rdtdf = Rdtdf[Rdtdf.columns[:-3]]
stopwords = ["i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now"]


['Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Healthcare', 'Industrials', 'Materials', 'Other', 'Real Estate', 'Technology', 'Telecom', 'Utilities']


In [128]:
# Prep
ETFdf = ETFdf[["Date", "ETF", "Sector", "Close"]]
ETFdf['Date']  = pd.to_datetime(ETFdf['Date'])
#ETFdf.set_index('Date', inplace=True)

Rdtdf = Rdtdf[['date_created', 'up_votes', 'down_votes', 'title']]
Rdtdf["title"] = Rdtdf["title"].str.split()
Rdtdf["date_created"] = pd.to_datetime(Rdtdf["date_created"])

import datetime
def removeStopWords(arr):
    for word in arr:
        if word in stopwords:
            arr.remove(word)  
    return arr

Rdtdf["title"].apply(removeStopWords)
#Rdtdf.set_index('date_created', inplace=True)
#index = pd.date_range(start='2008-1-25', end='2016-11-22', freq='7B', closed="left")

0                       [Scores, killed, Pakistan, clashes]
1                     [Japan, resumes, refuelling, mission]
2                        [US, presses, Egypt, Gaza, border]
3           [Jump-start, economy:, Give, health, care, all]
4         [Council, Europe, bashes, EU&UN, terror, black...
                                ...                        
509231    [Heil, Trump, :, Donald, Trump, alt-right, whi...
509232    [There, people, speculating, this, could, Made...
509233      [Professor, receives, Arab, Researchers, Award]
509234    [Nigel, Farage, attacks, response, Trump, amba...
509235    [Palestinian, wielding, knife, shot, dead, Wes...
Name: title, Length: 509236, dtype: object

In [129]:
startDate = Rdtdf.date_created[0]
# Make new column that has week index
ETFdf['week'] = ((ETFdf.Date - startDate).dt.days/7).astype(np.int)
Rdtdf['week'] = ((Rdtdf.date_created - startDate).dt.days/7).astype(np.int)

# Split dataframe into many dataframes by week index
alldf = Rdtdf.groupby(Rdtdf["week"])
framesArr = [] # put all dataframes in an array, because I don't know how else to store them
for i in range(len(alldf)): # loop to handle every dataframe seperately by week
    if i == 200: # Only the first 200 weeks for now
        break
    df = alldf.get_group(i)
    df.reset_index(drop=True, inplace=True)

    onehot = pd.get_dummies(pd.DataFrame(df['title'].sort_index().tolist()).stack()).sum(level=0)

    dfCombined = pd.concat([df, onehot], axis=1).drop(columns=['title'])
    framesArr.append(dfCombined)


In [131]:
framesArr[0][]

,date_created,up_votes,down_votes,week,-Times,40ft,Ad,Angela,Archbishop,Border,...,terror,the,thoughtless,threatens,told,town,tried,unveils,words,workers
0,2008-01-25,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2008-01-25,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2008-01-25,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2008-01-25,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2008-01-25,4,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2008-01-25,15,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
6,2008-01-25,5,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
7,2008-01-28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2008-01-28,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,2008-01-29,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
